In [1]:
import pandas as pd
import fnmatch
import cv2
import numpy as np
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import to_categorical
import pickle
import time
import keras
from sklearn.model_selection import train_test_split
import keras_metrics
import matplotlib.pyplot as plt


In [2]:
#AlexNet Architecture for OM estimation
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(300,300,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(3, activation='sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 73, 73, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 73, 73, 96)        384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 36, 36, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 36, 36, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 36, 36, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 17, 17, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 17, 384)       8

In [3]:
Lab_tested_data = pd.read_csv('C:\\Users\\Mahmood Yousaf\\Desktop\\FYP\\Final-Year-Project-Soil-Analysis-using-machine-learning\\Lab Results\\Soil_Lab_Results - Sheet1.csv')

In [4]:
print(len(Lab_tested_data))
#Getting sample id for each image with its lab value
Sample_ID = Lab_tested_data.iloc[:,0]
OM_Value = Lab_tested_data.iloc[:,4]

1572


In [5]:
def load_images_from_folder(folder):
    Labels = []
    image_counter = 0
    for filename in os.listdir(folder):
        x = filename.split("_")
        id = float(x[0])
        img = cv2.imread(os.path.join(folder,filename))
        width = 1000
        height = 1000
        dim = (width, height)
        # resize image
        img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        img = img[500:800,500:800] #Resizing the image
        kernel = np.array([[-1,-1,-1], 
                   [-1, 9,-1],
                   [-1,-1,-1]])
        img = cv2.filter2D(img, -1, kernel) # applying the sharpening kernel.
        Result = Sample_ID.isin([id])
        Result = Sample_ID[Result];
        if len(Result) >= 1:
            print(image_counter)
            image_counter = image_counter + 1
            if len(Result) == 2:  
                Result = Sample_ID[Sample_ID==Result.iloc[1]].index.tolist()
                Id_1_index = Result[0]
                Id_2_index = Result[1]
                #Average of both inner and surface image soil result 
                OM_lab_value = (OM_Value[Id_2_index] + OM_Value[Id_2_index]) / 2
                #Assigning classs number 0,1,2
                if OM_lab_value < 0.865:
                    Labels.append([filename,img,0])
                elif OM_lab_value >= 0.87 and OM_lab_value <= 1.29:
                    Labels.append([filename,img,1])
                elif OM_lab_value > 1.29:
                    Labels.append([filename,img,2])
            elif len(Result==1):
                Result = Sample_ID[Sample_ID==Result.iloc[1]].index.tolist()
                Id_1_index = Result[0]
                OM_lab_value = OM_Value[Id_1_index]
                #Assigning classs number 0,1,2
                if OM_lab_value < 0.865:
                    Labels.append([filename,img,0])
                elif OM_lab_value >= 0.87 and OM_lab_value <= 1.29:
                    Labels.append([filename,img,1])
                elif OM_lab_value > 1.29:
                    Labels.append([filename,img,2])
        else:
            print(len(Result))
            print("Hi i am here for testing")
            continue
    return Labels

In [6]:
#loading dataset 
dataset = load_images_from_folder('C:\\Users\\Mahmood Yousaf\\Desktop\\FYP\\Dataset RGB\\Training_data')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [7]:
#Preparing data for model
X = []
Y = []
for data in dataset:
    X.append(data[1])
    Y.append(data[2])
print(Y)
print(len(X))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [8]:
img_size = 300
X = np.array(X).reshape(-1, img_size, img_size, 3)
print(X[0].shape)
Y = to_categorical(Y)
print(Y[100])
print(X.shape)

(300, 300, 3)
[1. 0. 0.]
(1064, 300, 300, 3)


In [9]:
print(X.shape)
X_train, X_val, y_train, y_val = train_test_split(X, Y)
print(Y[522])

(1064, 300, 300, 3)
[1. 0. 0.]


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',keras_metrics.precision(),keras_metrics.recall(),keras_metrics.f1_score()])
history=model.fit(X_train, y_train, epochs=200, batch_size=32, verbose=1, validation_split=0.3)

Epoch 1/200


C:\Users\Mahmood Yousaf\AppData\Roaming\Python\Python38\site-packages\keras\engine\base_layer.py:1348: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '


18/18 [==============================] - 12s 356ms/step - loss: 44.9014 - accuracy: 0.9391 - precision: 0.9785 - recall: 0.9325 - f1_score: 0.9527 - val_loss: 152.7731 - val_accuracy: 0.9708 - val_precision: 0.9742 - val_recall: 0.9837 - val_f1_score: 0.9789
Epoch 2/200
18/18 [==============================] - 3s 180ms/step - loss: 4.4171 - accuracy: 0.9677 - precision: 0.9745 - recall: 0.9892 - f1_score: 0.9818 - val_loss: 9.6000 - val_accuracy: 0.9708 - val_precision: 0.9739 - val_recall: 0.9918 - val_f1_score: 0.9828
Epoch 3/200
18/18 [==============================] - 3s 180ms/step - loss: 1.6371 - accuracy: 0.9588 - precision: 0.9736 - recall: 0.9930 - f1_score: 0.9832 - val_loss: 59.4653 - val_accuracy: 0.9708 - val_precision: 0.9738 - val_recall: 0.9942 - val_f1_score: 0.9839
Epoch 4/200
18/18 [==============================] - 3s 180ms/step - loss: 2.3468 - accuracy: 0.9570 - precision: 0.9733 - recall: 0.9942 - f1_score: 0.9836 - val_loss: 3.1881 - val_accuracy: 0.9708 - val_p

18/18 [==============================] - 3s 185ms/step - loss: 0.1696 - accuracy: 0.9803 - precision: 0.9753 - recall: 0.9970 - f1_score: 0.9860 - val_loss: 0.3189 - val_accuracy: 0.9667 - val_precision: 0.9752 - val_recall: 0.9971 - val_f1_score: 0.9860
Epoch 22/200
18/18 [==============================] - 3s 182ms/step - loss: 0.2257 - accuracy: 0.9857 - precision: 0.9753 - recall: 0.9972 - f1_score: 0.9861 - val_loss: 0.2481 - val_accuracy: 0.9708 - val_precision: 0.9754 - val_recall: 0.9972 - val_f1_score: 0.9862
Epoch 23/200
18/18 [==============================] - 3s 180ms/step - loss: 0.2108 - accuracy: 0.9857 - precision: 0.9755 - recall: 0.9973 - f1_score: 0.9863 - val_loss: 0.2370 - val_accuracy: 0.9708 - val_precision: 0.9755 - val_recall: 0.9973 - val_f1_score: 0.9863
Epoch 24/200
18/18 [==============================] - 3s 180ms/step - loss: 0.1523 - accuracy: 0.9731 - precision: 0.9755 - recall: 0.9973 - f1_score: 0.9863 - val_loss: 0.3263 - val_accuracy: 0.9708 - val_pre

18/18 [==============================] - 3s 180ms/step - loss: 0.7500 - accuracy: 0.9588 - precision: 0.9759 - recall: 0.9980 - f1_score: 0.9868 - val_loss: 688.1164 - val_accuracy: 0.9708 - val_precision: 0.9758 - val_recall: 0.9980 - val_f1_score: 0.9868
Epoch 52/200
18/18 [==============================] - 3s 180ms/step - loss: 0.4328 - accuracy: 0.9731 - precision: 0.9758 - recall: 0.9980 - f1_score: 0.9868 - val_loss: 451.4034 - val_accuracy: 0.9708 - val_precision: 0.9759 - val_recall: 0.9980 - val_f1_score: 0.9868
Epoch 53/200
18/18 [==============================] - 3s 180ms/step - loss: 0.5143 - accuracy: 0.9731 - precision: 0.9759 - recall: 0.9980 - f1_score: 0.9868 - val_loss: 343.3562 - val_accuracy: 0.9708 - val_precision: 0.9760 - val_recall: 0.9980 - val_f1_score: 0.9868
Epoch 54/200
18/18 [==============================] - 3s 180ms/step - loss: 0.4164 - accuracy: 0.9677 - precision: 0.9760 - recall: 0.9979 - f1_score: 0.9869 - val_loss: 263.3356 - val_accuracy: 0.9708 -

18/18 [==============================] - 3s 181ms/step - loss: 0.1280 - accuracy: 0.9749 - precision: 0.9778 - recall: 0.9977 - f1_score: 0.9876 - val_loss: 2.0312 - val_accuracy: 0.9708 - val_precision: 0.9778 - val_recall: 0.9977 - val_f1_score: 0.9877
Epoch 82/200
18/18 [==============================] - 3s 181ms/step - loss: 0.2473 - accuracy: 0.9803 - precision: 0.9779 - recall: 0.9977 - f1_score: 0.9877 - val_loss: 1.8480 - val_accuracy: 0.9708 - val_precision: 0.9779 - val_recall: 0.9977 - val_f1_score: 0.9877
Epoch 83/200
18/18 [==============================] - 3s 181ms/step - loss: 0.1711 - accuracy: 0.9803 - precision: 0.9779 - recall: 0.9976 - f1_score: 0.9877 - val_loss: 1.8372 - val_accuracy: 0.9708 - val_precision: 0.9779 - val_recall: 0.9976 - val_f1_score: 0.9877
Epoch 84/200
18/18 [==============================] - 3s 181ms/step - loss: 0.1525 - accuracy: 0.9803 - precision: 0.9780 - recall: 0.9976 - f1_score: 0.9877 - val_loss: 1.5218 - val_accuracy: 0.9708 - val_pre

18/18 [==============================] - 3s 181ms/step - loss: 0.9930 - accuracy: 0.9677 - precision: 0.9785 - recall: 0.9976 - f1_score: 0.9879 - val_loss: 134.7937 - val_accuracy: 0.9708 - val_precision: 0.9785 - val_recall: 0.9976 - val_f1_score: 0.9879
Epoch 112/200
18/18 [==============================] - 3s 181ms/step - loss: 1.0919 - accuracy: 0.9749 - precision: 0.9785 - recall: 0.9976 - f1_score: 0.9879 - val_loss: 313.2626 - val_accuracy: 0.9708 - val_precision: 0.9785 - val_recall: 0.9976 - val_f1_score: 0.9880
Epoch 113/200
18/18 [==============================] - 3s 181ms/step - loss: 0.4200 - accuracy: 0.9695 - precision: 0.9785 - recall: 0.9976 - f1_score: 0.9879 - val_loss: 632.0859 - val_accuracy: 0.9708 - val_precision: 0.9785 - val_recall: 0.9976 - val_f1_score: 0.9880
Epoch 114/200
18/18 [==============================] - 3s 181ms/step - loss: 0.2400 - accuracy: 0.9677 - precision: 0.9785 - recall: 0.9976 - f1_score: 0.9879 - val_loss: 420.1327 - val_accuracy: 0.970

18/18 [==============================] - 3s 182ms/step - loss: 0.0861 - accuracy: 0.9785 - precision: 0.9788 - recall: 0.9977 - f1_score: 0.9881 - val_loss: 1.2793 - val_accuracy: 0.9708 - val_precision: 0.9788 - val_recall: 0.9977 - val_f1_score: 0.9882
Epoch 142/200
18/18 [==============================] - 3s 182ms/step - loss: 0.0899 - accuracy: 0.9910 - precision: 0.9788 - recall: 0.9977 - f1_score: 0.9882 - val_loss: 1.0063 - val_accuracy: 0.9708 - val_precision: 0.9788 - val_recall: 0.9977 - val_f1_score: 0.9882
Epoch 143/200
18/18 [==============================] - 3s 184ms/step - loss: 0.1305 - accuracy: 0.9857 - precision: 0.9788 - recall: 0.9977 - f1_score: 0.9882 - val_loss: 0.9851 - val_accuracy: 0.9708 - val_precision: 0.9788 - val_recall: 0.9978 - val_f1_score: 0.9882
Epoch 144/200
18/18 [==============================] - 3s 185ms/step - loss: 0.2456 - accuracy: 0.9767 - precision: 0.9788 - recall: 0.9978 - f1_score: 0.9882 - val_loss: 1.9561 - val_accuracy: 0.9708 - val_

18/18 [==============================] - 3s 183ms/step - loss: 0.0518 - accuracy: 0.9892 - precision: 0.9792 - recall: 0.9980 - f1_score: 0.9885 - val_loss: 1.6588 - val_accuracy: 0.9708 - val_precision: 0.9792 - val_recall: 0.9980 - val_f1_score: 0.9885
Epoch 172/200
18/18 [==============================] - 3s 181ms/step - loss: 0.0547 - accuracy: 0.9857 - precision: 0.9792 - recall: 0.9980 - f1_score: 0.9885 - val_loss: 0.9235 - val_accuracy: 0.9708 - val_precision: 0.9792 - val_recall: 0.9980 - val_f1_score: 0.9885
Epoch 173/200
18/18 [==============================] - 3s 181ms/step - loss: 0.0784 - accuracy: 0.9857 - precision: 0.9792 - recall: 0.9980 - f1_score: 0.9885 - val_loss: 0.9244 - val_accuracy: 0.9708 - val_precision: 0.9792 - val_recall: 0.9980 - val_f1_score: 0.9885
Epoch 174/200
18/18 [==============================] - 3s 181ms/step - loss: 0.0423 - accuracy: 0.9910 - precision: 0.9792 - recall: 0.9980 - f1_score: 0.9885 - val_loss: 0.7863 - val_accuracy: 0.9708 - val_

In [ ]:
model.evaluate(X_val,y_val)

In [ ]:
y_val[100]

In [ ]:
history.history.keys()

In [ ]:
print(history.history.keys())
plt.plot(history.history['precision'])
plt.plot(history.history['recall'])
plt.plot(history.history['f1_score'])
plt.title('Precision Recall')
plt.ylabel('Precision')
plt.xlabel('Recall')
plt.xlabel('accuracy')

plt.legend(['Precision', 'Recall','F1 Score'], loc='upper left')
plt.show()